In [1]:
import numpy as np

tang_file = np.load("tang.npz",allow_pickle=True)
tang_file.files

['ix2word', 'word2ix', 'data']

In [2]:
word2ix = tang_file['word2ix'].item()
idx2word = tang_file['ix2word'].item()

In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn
class PoemNet(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        """
            vocab_size：训练集合字典大小
            embedding_dim：word2vec的维度
            hidden_dim：LSTM的hidden_dim
        """
        super(PoemNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, self.hidden_dim,batch_first=True)

        self.fc = nn.Sequential(
            nn.Linear(self.hidden_dim,2048),
            nn.ReLU(),
            nn.Dropout(0.25),
            
            nn.Linear(2048,4096),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(4096,vocab_size),
        )

    def forward(self, input,hidden=None):
        """
            input：输入的诗词
            hidden：在生成诗词的时候需要使用，在pytorch中，，如果不指定初始状态h_0和C_0，则其
            默认为0.
            pytorch的LSTM的输出是(output,(h_n,c_n))。实际上，output就是h_1,h_2,……h_n
        """
        embeds = self.embeddings(input)
        batch_size, seq_len = input.size()
        if hidden is None:
            output, hidden = self.lstm(embeds)
        else:
            # h_0,c_0 = hidden
            output, hidden = self.lstm(embeds,hidden)
    
        output = self.fc(output)
        output = output.reshape(batch_size * seq_len, -1)
        output = F.log_softmax(output,dim=1)
        return output,hidden

/home/stephen/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
my_net = torch.load("model.h5").to(device)

生成一首诗 `<START>南楼夜已寂，暗鸟动林间。不见城郭事，沈沈唯四山。<EOP>'`

In [5]:
my_net.eval()

PoemNet(
  (embeddings): Embedding(8293, 200)
  (lstm): LSTM(200, 1024, batch_first=True)
  (fc): Sequential(
    (0): Linear(in_features=1024, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.25, inplace=False)
    (3): Linear(in_features=2048, out_features=4096, bias=True)
    (4): Dropout(p=0.2, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=4096, out_features=8293, bias=True)
  )
)

In [6]:
def generate_poem(my_words,max_len=128):
    '''
        根据前文my_words生成一首诗。max_len表示生成诗的最大长度。
    '''

    def __generate_next(idx,hidden=None):
        """
            根据input和hidden输出下一个预测词
        """
        input = torch.Tensor([idx]).view(1,1).long().to(device)
        output,hidden = my_net(input,hidden)
        return output,hidden

    # 初始化hidden状态
    output,hidden = __generate_next(word2ix["<START>"])
    my_words_len = len(my_words)
    result = []
    for word in my_words:
        result.append(word)
        # 积累hidden状态（h & c）
        output,hidden = __generate_next(word2ix[word],hidden)
    
    _,top_index = torch.max(output,1)

    word = idx2word[top_index[0].item()]

    result.append(word)

    for i in range(max_len-my_words_len):
        output,hidden = __generate_next(top_index[0].item(),hidden)

        _,top_index = torch.max(output,1)
        if top_index[0].item() == word2ix['<EOP>']: # 如果诗词已经预测到结尾
            break
        word = idx2word[top_index[0].item()]
        result.append(word)
    return "".join(result)

generate_poem("睡觉")

'睡觉寒炉火，晨钟坐中朝。炉烟沾煖露，池月静清砧。自有传心法，曾无住处传。不知尘世隔，一觉一壺秋。皎洁垂银液，浮航入绿醪。谁知旧邻里，相对似相亲。'

In [9]:
def acrostic_poetry(my_words):
    def __generate_next(idx,hidden=None):
        """
            根据input和hidden输出下一个预测词
        """
        input = torch.Tensor([idx]).view(1,1).long().to(device)
        output,hidden = my_net(input,hidden)
        return output,hidden

    def __generate(word,hidden):
        """
            根据word生成一句诗（以“。”结尾的话） 如根据床生成“床前明月光，凝是地上霜。”
        """
        generate_word = word2ix[word]
        sentence = []
        sentence.append(word)
        while generate_word != word2ix["。"]: 
            output,hidden = __generate_next(generate_word,hidden)
            _,top_index = torch.max(output,1)
            generate_word = top_index[0].item()
            sentence.append(idx2word[generate_word])
        # 根据"。"生成下一个隐状态。
        _,hidden = __generate_next(generate_word,hidden)
        return sentence,hidden

    _,hidden = __generate_next(word2ix["<START>"])
    result = []
    for word in my_words:
        sentence,hidden = __generate(word,hidden)
        result.append("".join(sentence))
    print("\n".join(result))

acrostic_poetry("滚去读书")

滚发初生光，三乘如太白。
去去冥冥没，冥茫寄天海。
读书三十年，手把棼琴策。
书罢华省郎，忧人惜凋病。
